<a href="https://colab.research.google.com/github/alirezakavianifar/machineLearning/blob/main/dqn_pytorch_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y xvfb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.13).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [3]:
!pip install gym \
gym[box2d] \
torchtext==0.9.0 \
pytorch-lightning==1.5.8 \
pyvirtualdisplay



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 4.2 MB/s 
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 0.11.0
    Uninstalling torchmetrics-0.11.0:
      Successfully uninstalled torchmetrics-0.11.0


In [4]:
from pyvirtualdisplay import Display
Display(visible=False, size=(1400, 900)).start()

In [5]:
import copy
import gym
import torch
import random

In [7]:
import copy
import gym
import torch
import random

import numpy as np
import torch.nn.functional as F

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import Tensor, nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW


from pytorch_lightning import LightningModule, Trainer

from gym.wrappers import RecordVideo, RecordEpisodeStatistics


In [10]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

In [12]:
def display_video(episode=0):
  video_file = open(f'/content/videos/rl-video-episode-{episode}.mp4', 'r+b').read()
  video_url = f'data:video/mp4;base64, {b64encode(video_file).decode()}'
  return HTML(f"<video width=600 controls><source src='{video_url}'></video>")

In [13]:
class DQN(nn.Module):

    def __init__(self, hidden_size, obs_size, n_action) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_action)
        )

    def forward(self, x):
        return self.net(x.float())


In [14]:
def epsilon_greedy(state, env, net, epsilon=0.0):
    if np.random.random() < epsilon:
        action = env.action_space.sample()
    else:
        state = torch.tensor([state]).to(device)
        q_values = net(state)
        _, action = torch.max(q_values, dim=1)
        action = int(action.item())
    return action

In [15]:
class ReplayBuffer:
    def __init__(self, capacity) -> None:
        self.buffer = deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)


In [16]:
class RLDataset(IterableDataset):
    def __init__(self, buffer, sample_size=200) -> None:
        self.buffer = buffer
        self.sample_size = sample_size

    def __iter__(self):
        for experience in self.buffer.sample(self.sample_size):
            yield experience

In [21]:
def create_environment(name):
    env = gym.make(name)
    env = RecordVideo(env, video_folder='./videos', episode_trigger=lambda x: x % 50 ==0)
    return env


env = create_environment('LunarLander-v2')

/usr/local/lib/python3.8/dist-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [22]:
class DeepQLearning(LightningModule):
    def __init__(self, env_name, policy=epsilon_greedy,
                 capacity=100_000, batch_size=256, lr=1e-3, hidden_size=128,
                 gamma=0.99, loss_fn=F.smooth_l1_loss, optim=AdamW,
                 eps_start=1.0, eps_end=0.15, eps_last_episode=100,
                 samples_per_epoch=10_000, syn_rate=10) -> None:
        super().__init__()
        self.env = create_environment(env_name)

        obs_size = self.env.observation_space.shape[0]
        n_action = self.env.action_space.n

        self.q_net = DQN(hidden_size, obs_size, n_action)

        self.target_q_net = copy.deepcopy(self.q_net)

        self.policy = policy
        self.buffer = ReplayBuffer(capacity=capacity)

        self.save_hyperparameters()

        while len(self.buffer) < self.hparams.samples_per_epoch:
            print(f'{len(self.buffer)} samples is experience buffer. filling...')
            self.play_episode(epsilon=self.hparams.eps_start)

        @torch.no_grad()
        def play_episode(self, policy=None, epsilon=0.):
            state = self.env.reset()
            done = False

            while not done:
                if policy:
                    action = policy(state, self.env,
                                    self.q_net, epsilon=epsilon)
                else:
                    action = self.env.action_space.sample()
                next_state, reward, done, info = self.env.step(action)
                exp = (state, action, reward, done, next-state)
                self.buffer.append(exp)
                state = next_state

        def forward(self, x):
            return self.q_net(x)

        def configure_optimizers(self):
            q_net_optimizer = self.hparams.optim(
                self.q_net.parameters(), lr=self.hparams.lr)
            return [q_net_optimizer]

        def train_dataloader(self):
            dataset = RLDataset(self.buffer, self.hparams.samples_per_epoch)
            dataloader = DataLoader(dataset=dataset,
                                    batch_size=self.hparams.batch_size)
            return dataloader

        def training_step(self, batch, batch_idx):
            states, actions, rewards, dones, next_states = batch
            actions = actions.unsqueeze(1)
            rewards = rewards.unsqueeze(1)
            done = dones.unsqueeze(1)

            state_action_values = self.q_net(states).gather(1, actions)
            next_action_values, _ = self.target_q_net(
                next_states).max(dim=1, keepdims=True)
            next_action_values[dones] = 0.0

            expected_state_action_values = rewards + \
                self.hparams.gamma * next_action_values

            loss = self.hparams.loss_fn(
                state_action_values, expected_state_action_values)
            self.log('episode/Q-Error', loss)
            return loss
        
        def training_epoch_end(self):
            
            epsilon = max(self.hparams.eps_end,
                          self.hparams.eps_start - self.current_epoch / self.hparams.eps_last_episode)
            
            self.play_episode(policy=self.policy, epsilon=epsilon)
            self.log('episode/Return', self.env.return_queue[-1])
            
            if self.current_epoch % self.hparams.sync_rate == 0:
                self.target_q_net.load_state_dict(self.q_net.state_dict())